In [1]:
import pandas as pd
import math 
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle

In [2]:
#leemos los datos
datos=pd.read_pickle('ALERCE_stamps_2020.pkl', compression='infer')

## Preprocesamiento de datos

### Cuadradas

In [3]:
#cantidad de imagenes no cuadradas:
list_of_indexes=[]
for i in range(len(datos['images'])):
    if len(datos['images'][i])!=len(datos['images'][i][0]):
        list_of_indexes.append(i)

cantidad=len(list_of_indexes)

sumando=0
#creamos este indice ya que al sacar un indice de la lista, todos los indices se resta uno
#si vamos del mas pequeño al mas grande
for i in list_of_indexes:
    #lo sacamos de la data:
    datos['images'].pop(i-sumando)
    datos['labels'].pop(i-sumando)
    datos['metadata'].pop(i-sumando)
    sumando+=1
        
print('# Imagenes no cuadradas: '+str(cantidad))
print('\n Nuevo largo de data: '+str(len(datos['images'])))

# Imagenes no cuadradas: 650

 Nuevo largo de data: 51594


### Recorte a 21

In [4]:
# Función para recortar centros de las imagenes
def recorte(imagen):
    inicio = imagen[21:42]
    recorte = np.zeros([21, 21, 3])
    for i in range(len(inicio)):
        recorte[i] = inicio[i][21:42]
    return recorte

In [5]:
#Recorte de todas las imagenes
dic={'im_21':[]}
for i in range(len(datos['images'])):
    dic['im_21'].append(recorte(datos['images'][i]))
datos.update(dic)

### Normalizar

In [6]:
def normalizar(matrix,matrix_zero,muestra):
    """ recibe matrix y la devuelve normalizada
    """
    maximo=np.amax(matrix[:,:,muestra])
    minimo=np.amin(matrix[:,:,muestra])
    matrix_zero[:,:,muestra]=(matrix[:,:,muestra]-minimo)/(maximo-minimo)
    return matrix_zero

In [7]:
#vamos viendo maximos y minimos, y normalizando

#matriz
dic={'images':[], 'im_21':[]}

contador=1
for i in range(len(datos['images'])):
    matrix_image=datos['images'][i]
    matrix21=datos['im_21'][i]
    matriz_zeros_paranormalizar=np.zeros((len(matrix_image),len(matrix_image[0]),3))
    matriz_zeros21=np.zeros((len(matrix21),len(matrix21[0]),3))
    #3 muestras
    for z in range(3):
        matriz_zeros_paranormalizar=normalizar(matrix_image, matriz_zeros_paranormalizar, z)
        matriz_zeros21=normalizar(matrix21, matriz_zeros21, z)
    dic['images'].append(matriz_zeros_paranormalizar)
    dic['im_21'].append(matriz_zeros21)
    contador+=1

datos.update(dic)
del(dic)
print('Normalización hecha con exito') 

Normalización hecha con exito


### Nan a 0

In [8]:
#Para imagenes de 63:
#vemos cuantos datos son NaN y lo cambiamos a 0

cantidad=0
#matriz
for i in range(len(datos['images'])):
    array_nan_index=np.argwhere(np.isnan(datos['images'][i]))
    for j in array_nan_index:
        index_fila=j[0]
        index_columna=j[1]
        index_muestra=j[2]
        datos['images'][i][index_fila][index_columna][index_muestra]=0
    cantidad+=len(array_nan_index)
        
print('# pixeles == NaN: '+str(cantidad))   
print('Cambiados a 0 con exito')

# pixeles == NaN: 74418750
Cambiados a 0 con exito


In [9]:
#Para imagenes de 21:
#vemos cuantos datos son NaN y lo cambiamos a 0

cantidad=0
#matriz
for i in range(len(datos['im_21'])):
    array_nan_index=np.argwhere(np.isnan(datos['im_21'][i]))
    for j in array_nan_index:
        index_fila=j[0]
        index_columna=j[1]
        index_muestra=j[2]
        datos['im_21'][i][index_fila][index_columna][index_muestra]=0
    cantidad+=len(array_nan_index)
        
print('# pixeles == NaN: '+str(cantidad))   
print('Cambiados a 0 con exito')

# pixeles == NaN: 3394818
Cambiados a 0 con exito


## Split sets

In [10]:
#Entrega lista con los indices de las instancias de cierta categoria de objeto astronomico
def get_index_cat(dic_datos, cat):
    index_list = []
    for i in range(len(dic_datos['labels'])):
        if dic_datos['labels'][i]==cat:
            index_list.append(i)
    return index_list

# Genera listas de índices para train, validation y test
def get_sets_index(dic_datos):
    val_index=np.array([])
    test_index=np.array([])
    train_index=np.array([])
    for k in range (5):
        L=get_index_cat(dic_datos, k)
        random.shuffle(L)
        test_index=np.append(test_index,L[0:200])
        val_index=np.append(val_index,L[200:300])

        if k == 1:
            for j in range (11):
                train_index=np.append(train_index,L[300:])
            train_index=np.append(train_index,L[300:486])
        else:
            train_index=np.append(train_index,L[300:])
            if k == 2:
                train_index=np.append(train_index,L[300:319])
            if k == 3:
                train_index=np.append(train_index,L[300:5243])                    
            if k == 4:
                train_index=np.append(train_index,L[300:4308])
                    
    random.shuffle(test_index)
    random.shuffle(val_index)
    random.shuffle(train_index)
    return train_index, val_index, test_index

In [11]:
#Crea diccionarios con los sets de datos
train_index, val_index, test_index = get_sets_index(datos) #obtenemos los indices
test_set={'images':[], 'im_21':[], 'labels':[], 'metadata':[]} #creacion diccionario con testing set
val_set={'images':[], 'im_21':[], 'labels':[], 'metadata':[]} #creacion diccionario con validation set
train_set={'images':[], 'im_21':[], 'labels':[], 'metadata':[]} #creacion diccionario con training set
for i in range(len(train_index)): #añade los elementos de los indices de train_index al train_set
    k=int(train_index[i])
    train_set['images'].append(datos['images'][k])
    train_set['im_21'].append(datos['im_21'][k])
    train_set['labels'].append(datos['labels'][k])
    train_set['metadata'].append(datos['metadata'][k])
for i in range(len(val_index)): #añade los elementos de los indices de val_index al val_set
    k=int(val_index[i])
    val_set['images'].append(datos['images'][k])
    val_set['im_21'].append(datos['im_21'][k])
    val_set['labels'].append(datos['labels'][k])
    val_set['metadata'].append(datos['metadata'][k])
for i in range(len(test_index)): #añade los elementos de los indices de test_index al test_set
    k=int(test_index[i])
    test_set['images'].append(datos['images'][k])
    test_set['im_21'].append(datos['im_21'][k])
    test_set['labels'].append(datos['labels'][k])
    test_set['metadata'].append(datos['metadata'][k])

In [16]:
# write python dict to a file
output = open('train_set.pkl', 'wb')
pickle.dump(train_set, output)
output.close()

output = open('val_set.pkl', 'wb')
pickle.dump(val_set, output)
output.close()

output = open('test_set.pkl', 'wb')
pickle.dump(test_set, output)
output.close()